# Using Compustat and NBER/USPTO Patent Database to construct R\&D spending measures

In this document I outline how I use Compustat and the NBER patent database to construct a measure of state-level of R&D spending.

## Constructing dataset

### Merging patents and inventors datasets

Code below:





In [3]:
install.packages('readstata13',repos='http://cran.us.r-project.org')

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [4]:
ptm <- proc.time()
library(readstata13)
source('stata_merge.R')





In [5]:
# Read in datasets
patents <- read.dta13('Raw/pat76_06_assg.dta')
names(patents)[names(patents) == 'appyear'] <- 'year'

inventors <- read.csv('Raw/inventor.csv')
names(inventors)[names(inventors) == 'Patent'] <- 'patent'


# Destring patent variable
inventors$patent = as.numeric(as.character(inventors$patent))



Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”

In [6]:
# Merge the datasets
#inventors_patents <- stata.merge(patents,inventors,"patent")
inventors_patents <- merge(patents,inventors,by = "patent")
rm(list = c("inventors","patents"))

In [7]:
proc.time() - ptm
dim(inventors_patents)
colnames(inventors_patents)
head(inventors_patents)


   user  system elapsed 
128.718   2.029 152.422 

[1] 7224831      41

[1] "patent"         "allcites"       "year"           "asscode"       
 [5] "assgnum"        "cat"            "cat_ocl"        "cclass"        
 [9] "country"        "ddate"          "gday"           "gmonth"        
[13] "gyear"          "hjtwt"          "icl"            "icl_class"     
[17] "icl_maingroup"  "iclnum"         "nclaims"        "nclass"        
[21] "nclass_ocl"     "pdpass"         "state"          "status"        
[25] "subcat"         "subcat_ocl"     "subclass"       "subclass1"     
[29] "subclass1_ocl"  "subclass_ocl"   "term_extension" "uspto_assignee"
[33] "Firstname"      "Lastname"       "Street"         "City"          
[37] "State"          "Country"        "Zipcode"        "Nationality"   
[41] "InvSeq"

patent,allcites,year,asscode,assgnum,cat,cat_ocl,cclass,country,ddate,⋯,uspto_assignee,Firstname,Lastname,Street,City,State,Country,Zipcode,Nationality,InvSeq
3930271,6,1974,2,1,6,6,2/161.4,US,NA,⋯,251415,HE CHUNG,KAHNG,,GLENVIEW,IL,US,60025,,1
3930272,0,1974,2,1,6,6,5/11,US,NA,⋯,246000,ROBERT J,BOUDREAU,,BEDFORD,PA,US,15522,,1
3930273,20,1975,2,1,6,6,5/430,US,NA,⋯,10490,ROBERT C,STERN,,SAINTLOUIS,MO,US,,,1
3930274,4,1974,1,NA,5,NA,114/354,,NA,⋯,0,HAROLD A,SYFRITT,,MANHATTAN BEACH,CA,US,90266,,1
3930275,1,1975,1,NA,6,6,12/142S,,NA,⋯,0,RICHARD,BAILIN,,ENGLEWOOD CLIFFS,NJ,US,07632,,1
3930276,3,1972,2,1,6,6,15/53.4,US,NA,⋯,243840,RUSSELL A,VAN BRAKEL,,HARSHAW,WI,US,54529,,1


#### Merging with dynass

In [8]:
dynass = read.dta13('Data/dynass_reshaped.dta')
head(dynass)

pdpass,spell,source,pdpco,begyr,gvkey,endyr,nspell,year
-19702,1,m2006,66448,1997,66448,2005,9,1997
-19702,1,m2006,66448,1997,66448,2005,9,1998
-19702,1,m2006,66448,1997,66448,2005,9,1999
-19702,1,m2006,66448,1997,66448,2005,9,2000
-19702,1,m2006,66448,1997,66448,2005,9,2001
-19702,1,m2006,66448,1997,66448,2005,9,2002


In [9]:
inventors_patents_dynass <- stata.merge(inventors_patents,dynass,c("pdpass","year"))

In [11]:
head(inventors_patents_dynass)
rm("inventors_patents")

pdpass,year,patent,allcites,asscode,assgnum,cat,cat_ocl,cclass,country,⋯,Nationality,InvSeq,spell,source,pdpco,begyr,gvkey,endyr,nspell,stata.merge.variable
-19702,1997,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,1,m2006,66448,1997,66448,2005,9,2
-19702,1998,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,1,m2006,66448,1997,66448,2005,9,2
-19702,1999,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,1,m2006,66448,1997,66448,2005,9,2
-19702,2000,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,1,m2006,66448,1997,66448,2005,9,2
-19702,2001,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,1,m2006,66448,1997,66448,2005,9,2
-19702,2002,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,1,m2006,66448,1997,66448,2005,9,2


In [12]:
tabulate(inventors_patents_dynass$stata.merge.variable)

[1] 4409704  249594 2815127